In [ ]:
from code_data_science import (
    data_table as dt,
    unique_dictionaries as ud,
    tree_data_grid,
)


def create_key(group_id=None, artifact_id=None, version=None):
    parts = []

    if group_id and artifact_id:
        parts.append(f"{group_id}.{artifact_id}")
    elif group_id:
        parts.append(group_id)
    elif artifact_id:
        parts.append(artifact_id)

    if version:
        parts.append(f"@{version}")

    return "".join(parts)


def dataframe_to_tree(df):
    df["groupId"].fillna("", inplace=True)
    df["version"].fillna("", inplace=True)
    df["dependencyGroupId"].fillna("", inplace=True)
    df["dependencyVersion"].fillna("", inplace=True)

    tree = ud.UniqueDictionaries()

    for _, row in df.iterrows():
        parentKey = f"{row['repositoryPath']}/{create_key(row['groupId'], row['artifactId'],row['version'])}"
        childKey = create_key(
            row["dependencyGroupId"],
            row["dependencyArtifactId"],
            row["dependencyVersion"],
        )

        tree.add(
            {
                "path": f"{parentKey}/{childKey}",
                "version": f"{row['dependencyVersion']}",
                "direct": f"{row['direct']}",
            }
        )

    return tree.to_list()


# dependency report recipe
df = dt.read_csv("../samples/dependency_tree_view.csv")
df = df.drop_duplicates(subset=["repositoryPath", "dependencyArtifactId"])
tree_data = dataframe_to_tree(df)

In [ ]:
tree_data_grid.display(
    tree_data,
    "Projects",
    [
        {"field": "version", "headerName": "Version", "minWidth": 200},
        {"field": "direct", "headerName": "Direct", "minWidth": 100},
    ],
)